# Tutorial 1.3: Introduction to Tracing

![](images/4_Notebook-13-Introduction-to-Tracing.png)

## LLM Observability for GenAI Applications

Welcome to the tracing section! This is where MLflow really shines for GenAI development. You'll learn how to gain complete visibility into your LLM applications.

### What You'll Learn
- What is tracing and why it matters for GenAI
- Understanding the trace data model (traces, spans, hierarchy)
- Automatic tracing with autologging
- Tracing multiple frameworks (OpenAI, LangChain)
- Viewing and analyzing traces in MLflow UI
- Debugging with traces uing Claude Code MLflow Assistant

### Prerequisites
- Completed Notebooks 1.1 and 1.2
- MLflow UI running
- OpenAI API key configured
- Or keys for your foundational model provider, e.g., Databricks

### Estimated Time: 20-25 minutes

---
## Step 1: What is Tracing?

### The Problem

Traditional experimenting tracking or logging isn't enough for LLM applications:

```python
# Traditional logging - hard to debug
print("Calling LLM...")
response = llm.generate(prompt)
print(f"Response: {response}")
# What happened inside? How long did it take? What was sent or received?
```

### The Solution: Distributed Tracing

Tracing captures the **complete execution flow** of your application:

```
Trace: RAG Application
├── Span 1: Embed Query [0ms - 200ms]
│   Input: "What is MLflow for GenAI?"
│   Output: [0.123, 0.456, ...]
│   
├── Span 2: Retrieve Documents [200ms - 350ms]
│   Input: [0.123, 0.456, ...]
│   Output: ["MLflow is...", "The platform..."]
│   
└── Span 3: Generate Response [350ms - 1500ms]
    Input: {query, documents}
    Output: "MLflow is an open source platform..."
    LLM: gpt-5-mini
    Tokens: 150
```

### Key Benefits

1. **Visibility**: See every step in your LLM workflow
2. **Performance**: Identify bottlenecks and latency issues
3. **Debugging**: Trace errors to their exact source
4. **Cost**: Track token usage per operation
5. **Quality**: Inspect inputs/outputs at each step

---
## Step 2: Understanding the Trace Data Model

### Core Concepts

**Trace**: A complete execution of an operation
- Represents one request or workflow
- Contains one or more spans
- Has a root span

**Span**: A single operation within a trace
- Has a start and end time
- Contains inputs and outputs
- Has metadata (model, tokens,latecy, etc.)
- Can have parent-child relationships

**Span Attributes**: Additional metadata
- Model name
- Token counts
- Temperature
- Custom attributes

### Span Types

MLflow defines standard span types:

- `CHAIN`: A sequence of operations
- `LLM`: Language model call
- `RETRIEVER`: Document retrieval
- `EMBEDDING`: Text embedding
- `TOOL`: Tool/function execution
- `AGENT`: Agent reasoning
- `PARSER`: Output parsing or generic intermediate parsing of an outcome 

### Hierarchy Example

```
TRACE (root)
│
└─ SPAN: Agent Executor (AGENT)
   │
   ├─ SPAN: Planning Step (LLM)
   │  └─ attributes: {model: gpt-5, tokens: 50}
   │
   ├─ SPAN: Tool Execution (TOOL)
   │  └─ attributes: {tool: search, query: "..."}
   │
   └─ SPAN: Final Response (LLM)
      └─ attributes: {model: gpt-5, tokens: 150}
```

---
## Step 3: Environment Setup

In [1]:
import mlflow
from dotenv import load_dotenv
from utils.clnt_utils import is_databricks_ai_gateway_client, get_databricks_ai_gateway_client, get_openai_client, get_ai_gateway_model_names

# Load environment
load_dotenv()

# Configure MLflow
mlflow.set_tracking_uri("http://localhost:5000")

use_databricks_provider = is_databricks_ai_gateway_client()
if use_databricks_provider:
    client = get_databricks_ai_gateway_client()
    model_name = get_ai_gateway_model_names()[0]
else:
    # Initialize OpenAI
    client = get_openai_client()
    model_name = "gpt-5-mini"

print("✅ Environment configured: using", "Databricks" if use_databricks_provider else "OpenAI", "client")
print(f"   MLflow version: {mlflow.__version__}")
print(f"   Tracking URI: {mlflow.get_tracking_uri()}")
print(f"   Using model: {model_name}")

✅ Environment configured: using OpenAI client
   MLflow version: 3.10.0rc0
   Tracking URI: http://localhost:5000
   Using model: gpt-5-mini


---
## Step 4: Your First Trace - Manual Example

Before we use autologging, let's understand what tracing captures by without autologging the information. This way we understand the difference and benefits of using autlogging. To some extent, we saw the benefits of autologgin in the previous notebook on experimental tracking LLM calls. So this is just a quick reminder in case you missed that tutorial.

In [2]:
# Create experiment for tracing examples
mlflow.set_experiment("06-tracing-introduction")

# Without tracing - basic call
print("\n📝 Making LLM call WITHOUT tracing...\n")

prompt = "Explain what distributed tracing is in one sentence."
response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": prompt}],
    temperature=1.0,
    max_completion_tokens=1000
)

print(f"Response: {response.choices[0].message.content}")
print("\n❌ What we DON'T know:")
print("   - Exact timing of the call")
print("   - Detailed request/response structure")
print("   - Easy way to correlate with other operations")
print("   - Visual representation of execution")

# check the trace in the UI
print("\n🔍 View trace in MLflow UI: http://localhost:5000")
print("   Navigate to: Traces tab")


2026/02/18 17:25:44 INFO mlflow.tracking.fluent: Experiment with name '06-tracing-introduction' does not exist. Creating a new experiment.



📝 Making LLM call WITHOUT tracing...

Response: Distributed tracing is a technique for recording and correlating timed operations (spans) across multiple services to reconstruct and analyze end-to-end request flows in distributed systems.

❌ What we DON'T know:
   - Exact timing of the call
   - Detailed request/response structure
   - Easy way to correlate with other operations
   - Visual representation of execution

🔍 View trace in MLflow UI: http://localhost:5000
   Navigate to: Traces tab


---
## Step 5: Automatic Tracing with OpenAI Autologging

Now let's enable tracing with a single line of code!

In [3]:
# Enable OpenAI autologging - THIS IS THE MAGIC LINE!
mlflow.openai.autolog()

print("✅ OpenAI autologging enabled")
print("   All OpenAI API calls will now be automatically traced!")

✅ OpenAI autologging enabled
   All OpenAI API calls will now be automatically traced!


In [4]:
# Make the same call - now it's automatically traced!
print("\n🔍 Making LLM call WITH tracing...\n")

response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": prompt}],
    temperature=1.0,
    max_completion_tokens=1000
)

print(f"Response: {response.choices[0].message.content}")
print("\n✅ What we NOW know:")
print("   ✓ Complete request details (model, messages, parameters)")
print("   ✓ Response content and metadata")
print("   ✓ Token usage (prompt, completion, total)")
print("   ✓ Timing information (latency)")
print("   ✓ All captured automatically!")
print("\n🔗 View trace in MLflow UI: http://localhost:5000")
print("   Navigate to: Traces tab")


🔍 Making LLM call WITH tracing...

Response: Distributed tracing is a technique for tracking and visualizing the path and timing of an individual request as it flows through multiple services or components in a distributed system by recording and correlating timed "spans" for each operation.

✅ What we NOW know:
   ✓ Complete request details (model, messages, parameters)
   ✓ Response content and metadata
   ✓ Token usage (prompt, completion, total)
   ✓ Timing information (latency)
   ✓ All captured automatically!

🔗 View trace in MLflow UI: http://localhost:5000
   Navigate to: Traces tab


Trace(trace_id=tr-4919148cf628cd8a9bb640baf35aea54)

### 🎉 What Just Happened?

With `mlflow.openai.autolog()`, MLflow automatically:

1. **Intercepted** the OpenAI API call
2. **Created** a trace with a unique ID
3. **Captured** all inputs (messages, model, parameters)
4. **Captured** all outputs (response, tokens, timing)
5. **Stored** everything in a structured format
6. **Made it available** in the MLflow UI

**Single line of code change required!**

---
## Step 6: Tracing Multiple Sequential Calls

Let's see how tracing helps with multi-step workflows. That is, multiple sequential call, each creating a trace of its own. This would be typical agentic workflow where an agent might make a series of sequentail calls.

In [5]:
# Simple multi-step workflow
print("\n🔄 Multi-step workflow with automatic tracing...\n")

# Step 1: Generate a topic
print("Step 1: Generating topic...")
topic_response = client.chat.completions.create(
    model=model_name,
    messages=[
        {
        "role": "system",
            "content": "You are a helpful AI expert who explains concepts clearly and concisely."
        },
        {
        "role": "user",
        "content": "Suggest one interesting AI topic for a blog post."
    }],
    temperature=1.0,
    max_completion_tokens=1000
)

# get the topic
topic = topic_response.choices[0].message.content
print(f"  Topic: {topic}")

# Step 2: Generate an outline
print("\nStep 2: Creating outline...")
outline_response = client.chat.completions.create(
    model=model_name,
    messages=[{
        "role": "user",
        "content": f"Create a 3-point outline for a blog post about: {topic}"
    }],
    temperature=1.0,
    max_completion_tokens=500
)
outline = outline_response.choices[0].message.content
print(f"  Outline: {outline[:100]}...")

# Step 3: Write the introduction
print("\nStep 3: Writing introduction...")
intro_response = client.chat.completions.create(
    model=model_name,
    messages=[{
        "role": "user",
        "content": f"Write a 4-sentence introduction paragraph for: {topic}"
    }],
    temperature=1.0,
    max_completion_tokens=2000
)
intro = intro_response.choices[0].message.content
print(f"  Introduction: {intro}")

print("\n✅ All three steps completed!")
print("\n📊 Total tokens used:")
total_tokens = (topic_response.usage.total_tokens +
                outline_response.usage.total_tokens +
                intro_response.usage.total_tokens)
print(f"   {total_tokens} tokens")

print("\n🔍 View in MLflow UI:")
print("   You'll see THREE separate traces, one for each call")
print("   Each trace shows timing, tokens, and complete I/O")


🔄 Multi-step workflow with automatic tracing...

Step 1: Generating topic...
  Topic: Topic: "Tool-Using AI Agents — How LLMs Become Task-Oriented Helpers"

One-sentence hook: Explore how large language models are being combined with external tools (search, code execution, APIs, browsers, sensors) and orchestration to form autonomous agents that can perform real-world tasks, and what that means for usefulness, reliability, and safety.

Why it’s interesting:
- Shows a concrete path from conversational models to practical automation (booking, data analysis, coding, research).
- Raises current technical questions (planning, tool selection, chaining, grounding) and evaluation challenges.
- Touches on important safety, privacy, and trust trade-offs as models gain ability to act.
- Lots of accessible demos and code repos to illustrate concepts for readers.

Suggested short outline for the post:
1. Opening example: a single prompt that triggers an agent to search the web, run a script, and s

[Trace(trace_id=tr-d6fa8cbc53a47322396496f4a0e0d783), Trace(trace_id=tr-cb6b647607a3166fddc2243f443b8305), Trace(trace_id=tr-5a8fab2b15557523da75ff995425b5d1)]

### 💡 Observation

Each OpenAI call creates a **separate trace**. 

(In the next notebook, we'll learn how to group related calls into a **single hierarchical trace** using manual instrumentation.)

---
## Step 7: Tracing with LangChain (Framework Integration)

MLflow supports automatic tracing for many frameworks, over 30+ Let's try a popular one LangChain!

LangChain is a Python/JS framework for building LLM-powered applications by providing reusable building blocks for prompts, models, tools/function-calling, retrieval (RAG), memory/state, evalution, and multi-step chains/agents.

Why/when to use it
Use LangChain when you want to:

**Standardize an LLM app**: consistent interfaces for prompt templates, chat models (e.g., ChatOpenAI), outputs/parsers, retries, and tracing.

**Build RAG**: connect documents → embeddings → vector store → retriever → prompt, with common patterns already implemented.

**Use tools/agents**: let the model call functions/tools and orchestrate multi-step workflows.

**Compose pipelines**: easily chain steps (prompt → model → parser) and swap components without rewriting glue code.

In [6]:
# Install LangChain if needed
!pip install langchain langchain-openai -q

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from utils.clnt_utils import get_langchain_chat_openai_client, get_databricks_ai_gateway_langchain_client

databricks_provider = is_databricks_ai_gateway_client()
# Enable LangChain autologging
mlflow.langchain.autolog()

print("✅ LangChain autologging enabled")
print("✅ Using", "Databricks AI Gateway" if databricks_provider else "OpenAI", "as provider")

✅ LangChain autologging enabled
✅ Using OpenAI as provider


In [9]:
# Create a simple LangChain chain
print("\n🔗 Creating and running LangChain chain with tracing...\n")

# Define prompt template
prompt_template = ChatPromptTemplate.from_template(
    "You are a {role}. Answer the following question: {question}"
)

# Create LLM
if databricks_provider:
    llm = get_databricks_ai_gateway_langchain_client(model_name, temperature=1.0)
else:
    llm = get_langchain_chat_openai_client(model_name, temperature=1.0)

# Create chain using the prompt template and the LLM. 
# the UNIX like Pipe operator | is used to chain the prompt template and the LLM.
chain = prompt_template | llm

# Run chain
response = chain.invoke({
    "role": "helpful AI assistant",
    "question": "What are the benefits of tracing in GenAI applications?"
})

print(f"Response: {response.content}")
print("\n✅ LangChain execution traced!")
print("\n🔍 In the trace, you'll see:")
print("   - Prompt template construction")
print("   - Variable substitution")
print("   - LLM invocation")
print("   - All as separate spans in a hierarchy!")


🔗 Creating and running LangChain chain with tracing...

Response: Short answer
Tracing — recording structured, correlated data about prompts, inputs, model calls, outputs, tool actions, retrievals, and timing — gives GenAI teams the visibility they need to operate, debug, govern and improve systems safely and efficiently.

Why tracing matters (key benefits)
- Faster debugging and root-cause analysis
  - See the exact prompt, model parameters, token stream, and intermediate tool calls that produced a bad output so you can reproduce and fix it.
- Reproducibility and experimentability
  - Versioned traces (model version, seed, config) let you re-run and compare experiments reliably.
- Observability and performance optimization
  - Latency, token counts and resource metrics let you spot bottlenecks, optimize prompt length, batching and cost.
- Provenance and hallucination mitigation
  - RAG/source traces show which documents or snippets the model used, making it easier to verify answers a

Trace(trace_id=tr-b7136987e2666f22357d6c4c8b2eb44e)

### 🎯 LangChain Tracing Benefits

With LangChain autologging, MLflow automatically traces:

- **Chain execution**: See how data flows through the chain
- **Prompt construction**: View how templates are filled
- **LLM calls**: Standard OpenAI tracing
- **Retrieval steps**: If using RAG components
- **Tool usage**: If using agents with tools

**All without manual instrumentation!**

---
## Step 8: Error Tracing and Debugging

Tracing is especially valuable when things go wrong. And with MLflow 3.9.0+, you can use Claude Code Assistant 
inpsect, debug, and suggest fixes for you.

We will quickly try this here....

In [ ]:
# Intentionally cause an error to see how it's traced
print("\n🐛 Demonstrating error tracing...\n")

try:
    # This will fail - invalid model name
    response = client.chat.completions.create(
        model="gpt-does-not-exist",
        messages=[{"role": "user", "content": "Hello"}]
    )
except Exception as e:
    print(f"❌ Error occurred: {type(e).__name__}")
    print(f"   Message: {str(e)[:100]}...")
    print("\n✅ Error was captured in trace!")
    print("\n🔍 In the MLflow UI:")
    print("   - Trace marked with error status")
    print("   - Error message visible")
    print("   - Stack trace preserved")
    print("   - Easy to identify failure point")
    print("   - Claude Code Assistant can inspect, debug, and suggest fixes!")


🐛 Demonstrating error tracing...

❌ Error occurred: NotFoundError
   Message: Error code: 404 - {'error': {'message': 'The model `gpt-does-not-exist` does not exist or you do not...

✅ Error was captured in trace!

🔍 In the MLflow UI:
   - Trace marked with error status
   - Error message visible
   - Stack trace preserved
   - Easy to identify failure point


Trace(trace_id=tr-27d14c787ea748e3767a9c44178494e0)

### Debugging with Traces

When your LLM application fails:

1. **Find the trace** in the UI (filter by error status)
2. **Identify the failing span** (marked red)
3. **Inspect inputs** that caused the error
4. **View error details** (message, type, stack trace)
5. **Fix and re-run** with full history preserved

---
## Step 9: Best Practices for Tracing

Let's review key best practices.

In [12]:
print("""
╔══════════════════════════════════════════════════════════════╗
║           Tracing Best Practices                             ║
╚══════════════════════════════════════════════════════════════╝

✅ DO:

1. Enable autologging early in development
   - Easier to debug from the start
   - Understand performance characteristics early

2. Use tracing in all environments
   - Development: Full tracing for debugging
   - Staging: Trace all requests
   - Production: Sample or trace all (depending on volume)

3. Combine tracing with experiment tracking
   - Traces give you the "how"
   - Experiments give you the "what" and "why"

4. Review traces regularly
   - Look for performance bottlenecks
   - Identify expensive operations
   - Understand failure patterns

5. Use multiple frameworks
   - MLflow supports 30+ integrations
   - Pick the best tool for each task

❌ DON'T:

1. Ignore traces until there's a problem
   - Proactive monitoring prevents issues

2. Log sensitive data in traces
   - PII, credentials, confidential info
   - Use data masking if needed

3. Assume autologging captures everything
   - Custom operations need manual instrumentation
   - We'll cover this in the next notebook!

4. Over-rely on individual traces
   - Look at aggregate patterns
   - Use traces + metrics together

💡 PRO TIPS:

- Set up trace sampling in high-volume production
- Archive old traces to manage storage
- Use MLflow UI to inspect traces and metrics
- Use Claude Code Assistant to inspect, debug, and suggest fixes

""")


╔══════════════════════════════════════════════════════════════╗
║           Tracing Best Practices                             ║
╚══════════════════════════════════════════════════════════════╝

✅ DO:

1. Enable autologging early in development
   - Easier to debug from the start
   - Understand performance characteristics early

2. Use tracing in all environments
   - Development: Full tracing for debugging
   - Staging: Trace all requests
   - Production: Sample or trace all (depending on volume)

3. Combine tracing with experiment tracking
   - Traces give you the "how"
   - Experiments give you the "what" and "why"

4. Review traces regularly
   - Look for performance bottlenecks
   - Identify expensive operations
   - Understand failure patterns

5. Use multiple frameworks
   - MLflow supports 30+ integrations
   - Pick the best tool for each task

❌ DON'T:

1. Ignore traces until there's a problem
   - Proactive monitoring prevents issues

2. Log sensitive data in traces
   - P

---
## Summary

In this notebook, you learned:

1. What tracing is and why it's critical for GenAI
2. The trace data model (traces, spans, hierarchy)
3. Automatic tracing with `mlflow.openai.autolog()`
4. How to view and analyze traces in the MLflow UI
5. Tracing multi-step workflows
6. Framework integration (LangChain example)
7. Error tracing and debugging
8. Best practices for production use

### Key Takeaways

- **One line of code** enables complete observability
- **Automatic capture** of inputs, outputs, and metadata
- **Framework agnostic** - works with OpenAI, LangChain, LlamaIndex, etc.
- **Essential for debugging** complex LLM workflows
- **Production-ready** with sampling and filtering

### What's Next?

**Notebook 1.4: Manual Tracing and Advanced Observability**

Learn how to:
- Create custom spans with `@mlflow.trace`
- Add custom attributes to spans
- Build hierarchical traces for complex workflows
- Trace RAG pipelines with retrieval and generation steps
- Trace agentic workflows with tool usage
- Debug production issues with advanced techniques